# Подсчет доли дубликатов в потоке поисковых запросов

In [199]:
import string
import random

def stream():
    while True:
        userid = ''.join(random.sample(string.digits, 4))
        text = ''.join(random.sample(string.ascii_lowercase, 20))
        yield userid, text
        if random.random() < 0.05:
            yield userid, text
        
def hash_query(text, hash_size=100000):
    result = sum(i ** ord(c) for i, c in enumerate(text, 2))
    return result % hash_size

In [267]:
import multiprocessing

SAMPLE = 0.2


from collections import defaultdict
counter_query = defaultdict(int)
counter_users = defaultdict(int)
counter_all = defaultdict(int)

for i, query in enumerate(stream()):

    text_hash = query[1]
    
    # все запросы пользователей
    counter_all[text_hash] += 1
    
    # учитываем запросы от 30% пользователей
    if hash_query(query[0]) % 1000 < 300:
        counter_users[text_hash] += 1
        
    # учитываем только SAMPLE% запросов
    if random.random() < SAMPLE:  # [0..1)
        counter_query[text_hash] += 1

    if i > 50000:
        break

In [268]:
# семплирование запросов
x_query = sum(1 for v in counter_query.values() if v == 1)
d_query = sum(1 for v in counter_query.values() if v == 2)

d = d_query / (SAMPLE**2)
queries = (x_query + d_query) / SAMPLE

print(1. * d_query / (x_query + d_query))
print(1. * d / queries)

0.010044070923439582
0.0502203546171979


In [269]:
# по всей выборке
x_all = sum(1 for v in counter_all.values() if v == 1)
d_all = sum(1 for v in counter_all.values() if v == 2)
1.* d_all / (x_all + d_all)

0.05070499485175146

In [270]:
# семплирование пользователей
x_users = sum(1 for v in counter_users.values() if v == 1)
d_users = sum(1 for v in counter_users.values() if v == 2)
1.* d_users / (x_users + d_users)

0.050684738105322605